In [1]:
#!/usr/bin/env python
# coding: utf-8

#===============================================================================
#  Tom Paskhalis and Kevin Aslett, 2022-03-21
#
#  Retrieve US data from SS1 Presto database
# 
#===============================================================================

import pandas as pd
import os
from fbri.private.sql.query import execute

In [2]:
#Name databases that we are querying:
DATABASE = "fbri_prod_private"
URL_TABLE = "erc_condor_url_attributes_dp_final_v3"
BD_TABLE = "erc_condor_url_breakdowns_dp_clean_partitioned_v2"

In [3]:
#Build function that produces data for a "target_country"
def build_sql_query_attributes(target_country):
    """Build an SQL query for retrieving URL attributes
    
    Takes string containing ISO 3166-1 alpha-2 letter country code as an input.
    Returns formed SQL query as an f-string.
    """
    sql_query_attributes = f"""
        SELECT
            urlattr.url_rid,
            urlattr.clean_url,
            urlattr.full_domain,
            urlattr.parent_domain,
            urlattr.first_post_time,
            urlattr.first_post_time_unix,
            urlattr.spam_usr_feedback,
            urlattr.false_news_usr_feedback,
            urlattr.hate_speech_usr_feedback,
            urlattr.public_shares_top_country
        FROM
            {DATABASE}.{URL_TABLE} urlattr
        WHERE
            urlattr.public_shares_top_country = '{target_country}'
    """
    
    return sql_query_attributes

In [4]:
#Set the list of first post time so we can pull by month across January, 2017- June, 2020:
Lower_first_post_time =         [0,
                        1485910800,
                        1488330000,
                        1491008400,
                        1493600400,
                        1496278800,
                        1501549200,
                        1504227600,
                        1506819600,
                        1509498000,
                        1512090000,
                        1514764800,
                        1517461200,
                        1519880400,
                        1522555200,
                        1525147200,
                        1527825600,
                        1530417600,
                        1533096000,
                        1535774400,
                        1538366400,
                        1541044800,
                        1543622400,
                        1546300800,
                        1548982800,
                        1551402000,
                        1554080400,
                        1556672400,
                        1559350800,
                        1561942800,
                        1564621200,
                        1567299600,
                        1569891600,
                        1572570000,
                        1575162000,
                        1577840400,
                        1580518800,
                        1583024400,
                        1585702800,
                        1588294800]


Upper_first_post_time =[1485910800,
                        1488330000,
                        1491008400,
                        1493600400,
                        1496278800,
                        1501549200,
                        1504227600,
                        1506819600,
                        1509498000,
                        1512090000,
                        1514764800,
                        1517461200,
                        1519880400,
                        1522555200,
                        1525147200,
                        1527825600,
                        1530417600,
                        1533096000,
                        1535774400,
                        1538366400,
                        1541044800,
                        1543622400,
                        1546300800,
                        1548982800,
                        1551402000,
                        1554080400,
                        1556672400,
                        1559350800,
                        1561942800,
                        1564621200,
                        1567299600,
                        1569891600,
                        1572570000,
                        1575162000,
                        1577840400,
                        1580518800,
                        1583024400,
                        1585702800,
                        1588294800,
                        1590973200]


Upper_month = ['2017-04',
                        '2017-05',
                        '2017-06',
                        '2017-07',
                        '2017-08',
                        '2017-09',
                        '2017-10',
                        '2017-11',
                        '2017-12',
                        '2018-01',
                        '2018-02',
                        '2018-03',
                        '2018-04',
                        '2018-05',
                        '2018-06',
                        '2018-07',
                        '2018-08',
                        '2018-09',
                        '2018-10',
                        '2018-11',
                        '2018-12',
                        '2019-01',
                        '2019-02',
                        '2019-03',
                        '2019-04',
                        '2019-05',
                        '2019-06',
                        '2019-07',
                        '2019-08',
                        '2019-09',
                        '2019-10',
                        '2019-11',
                        '2019-12',
                        '2020-01',
                        '2020-02',
                        '2020-03',
                        '2020-04',
                        '2020-05',
                        '2020-06',
                        '2020-07',
                        '2020-08',
                        '2020-09']


Lower_Month = ['2016-12',
                        '2017-01',
                        '2017-02',
                        '2017-03',
                        '2017-04',
                        '2017-05',
                        '2017-06',
                        '2017-07',
                        '2017-08',
                        '2017-09',
                        '2017-10',
                        '2017-11',
                        '2017-12',
                        '2018-01',
                        '2018-02',
                        '2018-03',
                        '2018-04',
                        '2018-05',
                        '2018-06',
                        '2018-07',
                        '2018-08',
                        '2018-09',
                        '2018-10',
                        '2018-11',
                        '2018-12',
                        '2019-01',
                        '2019-02',
                        '2019-03',
                        '2019-04',
                        '2019-05',
                        '2019-06',
                        '2019-07',
                        '2019-08',
                        '2019-09',
                        '2019-10',
                        '2019-11',
                        '2019-12',
                        '2020-01',
                        '2020-02',
                        '2020-03',
                        '2020-04',
                        '2020-05']


In [5]:
#Build function that produces data for a "target_country"
def build_sql_queries_reactions(target_country,i):
    """Build SQL query for retrieving aggreggated users' reactions before 
    """
    sql_query_reactions_1 = f"""
        WITH aggregated_urlbds AS
            (SELECT
                urlbd.url_rid AS url_rid,
                SUM(urlbd.views) AS total_views,
                SUM(urlbd.clicks) AS total_clicks,
                SUM(urlbd.shares) AS total_shares,
                SUM(urlbd.comments) AS total_comments,
                SUM(urlbd.share_without_clicks) AS total_shares_without_clicks,
                SUM(urlbd.likes) AS total_likes,
                SUM(urlbd.loves) AS total_loves,
                SUM(urlbd.hahas) AS total_hahas,
                SUM(urlbd.wows) AS total_wows,
                SUM(urlbd.sorrys) AS total_sorrys,
                SUM(urlbd.angers) AS total_angers
            FROM {DATABASE}.{BD_TABLE} urlbd
            WHERE date_parse(year_month, '%Y-%m') < date_parse('{Upper_month[i]}', '%Y-%m') AND date_parse(year_month, '%Y-%m') > date_parse('{Lower_Month[i]}', '%Y-%m')
            GROUP BY urlbd.url_rid)
        SELECT
            urlattr.url_rid,
            aggregated_urlbds.total_views,
            aggregated_urlbds.total_clicks,
            aggregated_urlbds.total_shares,
            aggregated_urlbds.total_comments,
            aggregated_urlbds.total_shares_without_clicks,
            aggregated_urlbds.total_likes,
            aggregated_urlbds.total_loves,
            aggregated_urlbds.total_hahas,
            aggregated_urlbds.total_wows,
            aggregated_urlbds.total_sorrys,
            aggregated_urlbds.total_angers
        FROM
            aggregated_urlbds JOIN
            {DATABASE}.{URL_TABLE} urlattr
                ON aggregated_urlbds.url_rid = urlattr.url_rid
        WHERE
            urlattr.public_shares_top_country = '{target_country}' AND
            urlattr.first_post_time_unix<={Upper_first_post_time[i]} AND
            urlattr.first_post_time_unix<{Upper_first_post_time[i]}
        """
    
    return sql_query_reactions_1

In [6]:
sql_query_attributes = build_sql_query_attributes('US')

In [7]:
filepath_attributes = "SS1_US_URL_attributes.tsv"
result = execute(sql_query_attributes, filepath_attributes)

[NOTICE] 2 output(s) filtered out

In [17]:
for x in range(0,len(Lower_first_post_time)):
    print(str(x+1) + '/' + str(len(Lower_first_post_time)))
    filepath_reactions = "SS1_US_reactions_" + str(x+1) + ".tsv"
    sql_queries_reactions = build_sql_queries_reactions('US',x)
    result = execute(sql_queries_reactions, filepath_reactions)
    #Create function that merges CSVs
    #Create function that deletes old CSVs

[NOTICE] 1 output(s) filtered out